In [14]:
import keras
import numpy as np
import numpy
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout
from keras.optimizers import Adam

In [17]:
base_dir = '../data/'

In [5]:
full_df = pd.read_csv(base_dir+'train_1.csv', nrows=100)
dates = [c for c in full_df.columns if c !='Page']
val_dates = dates[-61:]
val = full_df[['Page']+val_dates]
train = full_df.drop(val_dates, axis=1)
filled_train = train.fillna(0)

In [6]:
X = filled_train.drop('Page', axis=1).values
y = val.fillna(0).drop('Page', axis=1).values

In [7]:
print(X.shape, y.shape)

(100, 489) (100, 61)


## Preprocessing
 * scale each time series to have 0 mean and unit variance

In [8]:
# scaler = MinMaxScaler(feature_range=(0, 1))
scaler = StandardScaler()
# discussion on which scaler to use here: https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/38274
# each ts should have 0 mean and unit variance
# since the time series are the 'features' being scaled, transpose first
X = scaler.fit_transform(X.T).T
print(X.shape)
assert(np.isclose(np.mean(X[0]),0))
# input shape: samples, timesteps, features
X = X.reshape(X.shape[0], X.shape[1], 1)
print(X.shape)
np.max(X)

(100, 489)
(100, 489, 1)


22.010761664611856

In [18]:
DEFAULT_SEED = 123456
n_pred = 60 # number of values to predict
n_cond = 60 # number of values to condition on

class DataProvider(object):
    """Generic data provider."""

    def __init__(self, inputs, targets, batch_size, max_num_batches=-1,
                 shuffle_order=True, rng=None):
        """Create a new data provider object.

        Args:
            inputs (ndarray): Array of data input features of shape
                (num_data, input_dim).
            targets (ndarray): Array of data output targets of shape
                (num_data, output_dim) or (num_data,) if output_dim == 1.
            batch_size (int): Number of data points to include in each batch.
            max_num_batches (int): Maximum number of batches to iterate over
                in an epoch. If `max_num_batches * batch_size > num_data` then
                only as many batches as the data can be split into will be
                used. If set to -1 all of the data will be used.
            shuffle_order (bool): Whether to randomly permute the order of
                the data before each epoch.
            rng (RandomState): A seeded random number generator.
        """
        self.inputs = inputs
        self.targets = targets
        if batch_size < 1:
            raise ValueError('batch_size must be >= 1')
        self._batch_size = batch_size
        if max_num_batches == 0 or max_num_batches < -1:
            raise ValueError('max_num_batches must be -1 or > 0')
        self._max_num_batches = max_num_batches
        self._update_num_batches()
        self.shuffle_order = shuffle_order
        self._current_order = np.arange(inputs.shape[0])
        if rng is None:
            rng = np.random.RandomState(DEFAULT_SEED)
        self.rng = rng
        self.new_epoch()

    @property
    def batch_size(self):
        """Number of data points to include in each batch."""
        return self._batch_size

    @batch_size.setter
    def batch_size(self, value):
        if value < 1:
            raise ValueError('batch_size must be >= 1')
        self._batch_size = value
        self._update_num_batches()

    @property
    def max_num_batches(self):
        """Maximum number of batches to iterate over in an epoch."""
        return self._max_num_batches

    @max_num_batches.setter
    def max_num_batches(self, value):
        if value == 0 or value < -1:
            raise ValueError('max_num_batches must be -1 or > 0')
        self._max_num_batches = value
        self._update_num_batches()

    def _update_num_batches(self):
        """Updates number of batches to iterate over."""
        # maximum possible number of batches is equal to number of whole times
        # batch_size divides in to the number of data points which can be
        # found using integer division
        possible_num_batches = self.inputs.shape[0] // self.batch_size
        if self.max_num_batches == -1:
            self.num_batches = possible_num_batches
        else:
            self.num_batches = min(self.max_num_batches, possible_num_batches)

    def __iter__(self):
        """Implements Python iterator interface.

        This should return an object implementing a `next` method which steps
        through a sequence returning one element at a time and raising
        `StopIteration` when at the end of the sequence. Here the object
        returned is the DataProvider itself.
        """
        return self

    def new_epoch(self):
        """Starts a new epoch (pass through data), possibly shuffling first."""
        self._curr_batch = 0
        if self.shuffle_order:
            self.shuffle()

    def reset(self):
        """Resets the provider to the initial state."""
        inv_perm = np.argsort(self._current_order)
        self._current_order = self._current_order[inv_perm]
        self.inputs = self.inputs[inv_perm]
        self.targets = self.targets[inv_perm]
        self.new_epoch()

    def shuffle(self):
        """Randomly shuffles order of data."""
        perm = self.rng.permutation(self.inputs.shape[0])
        self._current_order = self._current_order[perm]
        self.inputs = self.inputs[perm]
        self.targets = self.targets[perm]
        if hasattr(self, 'track_ids'):
            self.track_ids = self.track_ids[perm]

    def next(self):
        """Returns next data batch or raises `StopIteration` if at end."""
        if self._curr_batch + 1 > self.num_batches:
            # no more batches in current iteration through data set so start
            # new epoch ready for another pass and indicate iteration is at end
            self.new_epoch()
            raise StopIteration()
        # create an index slice corresponding to current batch number
        batch_slice = slice(self._curr_batch * self.batch_size,
                            (self._curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self._curr_batch += 1
        return inputs_batch, targets_batch

    # Python 3.x compatibility
    def __next__(self):
        return self.next()
    
class MultiTSDataProvider(DataProvider):
    
    def __init__(self, which_set='train', batch_size=100, max_num_batches=-1,
                 shuffle_order=True, rng=None, n_pred=60, n_cond=60, stride_length=10,
                 scaler=StandardScaler(), base_dir = '../data/', n_ts=100):
        self.n_pred = n_pred # number of values to predict
        self.n_cond = n_cond # number of values on which to condition predictions
        full_df = pd.read_csv(base_dir+'train_1.csv', nrows=n_ts)
        dates = [c for c in full_df.columns if c !='Page']
        val_dates = dates[-n_pred:]
        self.val_dates = val_dates
        if which_set == 'train':
            inputs = full_df.drop(['Page'] + val_dates, axis=1).fillna(0).values
        elif which_set == 'val':
            inputs = full_df[val_dates].fillna(0).values
          # each ts should have 0 mean and unit variance
        if scaler:
            inputs = scaler.fit_transform(inputs)
            print(np.max(inputs))
        print(inputs.shape)
        
        window_length = n_cond + n_pred
        n_windows = int(np.floor(np.divide(inputs.shape[1] - window_length,
                                           stride_length) + 1))
        start_index = 0
        window_array = np.ndarray((inputs.shape[0], n_windows, window_length))
        for i in range(n_windows):
            window_array[:,i,:] = inputs[:, start_index:start_index+window_length]
            start_index += stride_length
        print(window_array.shape)
        window_array = window_array.reshape((-1,window_length,1))
        print(window_array.shape)
        print('{} overlapping windows of length {} in training date range'.format(n_windows, window_length))
#         inputs = w.reshape((-1, window_length, inputs.shape[-1])) # reshape inputs to (n_sample, length, n_features)
        inputs = np.pad(window_array[:,:-1,:],((0,0), (1,0), (0,0)),mode='constant')
        targets = inputs[:,self.n_cond:,:]
#         inputs, targets = inputs[:,:self.n_cond,:], inputs[:,self.n_cond:,:]
        inputs = inputs.reshape((inputs.shape[0], -1))
        targets = targets.reshape((targets.shape[0], -1))
        print(inputs.shape, targets.shape)
        super(MultiTSDataProvider, self).__init__(
            inputs, targets, batch_size, max_num_batches, shuffle_order, rng)

In [10]:
train_data = MultiTSDataProvider(scaler=StandardScaler())
a = train_data.next()[0]
np.max(a)

9.86570145996
(100, 490)
(100, 38, 120)
(3800, 120, 1)
38 overlapping windows of length 120 in training date range
(3800, 120) (3800, 60)


9.5193152455075101

### Loss Function:

Modified SMAPE. Note that using MAE gives not much different results, but easier and faster to train, so I recommend MAE for starters. (https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/38337)

### Output:

You can predict one day, refit the model with previous days + predicted day, predict next day, etc, but it would be too slow and inefficient. The real power of RNN's is that you can build generative model, and predict all 60 days at once. (https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/38337)

ONLY PREDICT VALUES FOR A SINGLE TIME SERIES AT ONCE (c.f. the neural network eqn. 1 which outputs hidden rnn states for a single time series at a time, which are then combined to form the joint likelihood above) - DIFFERENT TIME SERIES ARE USED TO GENERATE DIFFERENT TRAINING EXAMPLES; SINGLE MODEL IS TRAINED ON ALL OF THEM.

a substantial amount
of data on past behavior of similar, related time series can be leveraged for making a forecast for an
individual time series. Using data from related time series not only allows fitting more complex (and
hence potentially more accurate) models without overfitting

https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/

Simplest model is probably some kind of ar

### Training examples

Secondly, due to the imbalance in the data, a stochastic optimization procedure that picks training
instances uniformly at random will visit the small number time series with a large scale very infrequently,
which result in underfitting those time series. This could be especially problematic in the
demand forecasting setting, where high-velocity items can exhibit qualitatively different behavior
than low-velocity items, and having an accurate forecast for high-velocity items might be more important
for meeting certain business objectives. To counteract this effect, we sample the examples
non-uniformly during training. In particular, in our weighted sampling scheme, the probability of
selecting a window from an example with scale νi is proportional to νi.

### Tensorflow seq2seq
[github seq2seq helpers](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/legacy_seq2seq/python/ops/seq2seq.py)
[ex. model built using above](https://github.com/tensorflow/models/blob/master/tutorials/rnn/translate/seq2seq_model.py)
[keras lib](https://github.com/farizrahman4u/seq2seq)
[new api (1.3) seq2seq](https://medium.com/@ilblackdragon/tensorflow-sequence-to-sequence-3d9d2e238084)
[seq2seq expts](https://github.com/raindeer/seq2seq_experiments/blob/master/model.py#L81)
* use tied_rnn_seq2seq if using tf
* will need to write custom loss func
* not sure the keras library will work because although we are sort of using encoder-decoder, we're not really outputting a 'context vector', just transferring the hidden state
* to pass predicted values in as inputs when decoding: https://stackoverflow.com/questions/38050333/how-to-predict-a-simple-sequence-using-seq2seq-from-tensorflow (although there is an option just to use the true values during training, which is what the amazon paper does).

The initial state
of the encoder hi,0 as well as zi,0 are initialized to zero

In [22]:
import tensorflow as tf

lstm_size = 100
n_cond = 60
n_pred = 60

def compute_next_input(output, i):
    p = tf.reshape(tf.matmul(output, weights), (-1,)) + bias
    return tf.reshape(p,(-1,1))

looper = lambda output, i: tf.reshape(tf.reshape(tf.matmul(output, weights), (-1,)) + bias, (-1,1))

graph = tf.Graph()
with graph.as_default():
    cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    encoder_inputs = []
    decoder_inputs = []
    targets = []
    for i in range(n_cond):
        # individual inputs are single numbers 
        encoder_inputs.append(tf.placeholder(tf.float32, shape=[None,1], name="encoder{0}".format(i)))
    for i in range(n_pred):
        decoder_inputs.append(tf.placeholder(tf.float32, shape=[None,1], name="decoder{0}".format(i)))
    for i in range(n_pred):
        targets.append(tf.placeholder(tf.float32, shape=[None], name="target{0}".format(i)))
        
#     targets = [decoder_inputs[i+1] for i in range(len(decoder_inputs)-1)] # targets are next inputs
    outputs, states = tf.contrib.legacy_seq2seq.tied_rnn_seq2seq(encoder_inputs, decoder_inputs, cell)
    # to do val, need to define a loop function
    # outputs is a list where each thing has shape (batch_size,n_lstm)
    # loop_function will need to apply a linear transform
    preds = []
    losses = []
    for output, target in zip(outputs, targets):
        bias = tf.Variable(initial_value=1.0)
        weights = tf.Variable(tf.truncated_normal(
            [lstm_size, 1], stddev=2. / (lstm_size + 1)**0.5), 
        'weights')
#         weights = tf.Variable(tf.ones([lstm_size,1]))
        pred = tf.reshape(tf.matmul(output, weights), (-1,)) + bias
        preds.append(pred) # each pred is shape [batch_size]
        losses.append(tf.reduce_mean(tf.abs(pred-target))) # error for a given output position averaged over the batch
    
    mae = tf.reduce_mean(losses) # compute error averaged over the timesteps (and batch samples)
#     mae = tf.metrics.mean_absolute_error(targets, outputs)
    train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(mae)
    init_op = tf.global_variables_initializer()

In [23]:
train_data = MultiTSDataProvider(n_ts=1000)

sess = tf.Session(graph=graph)
sess.run(init_op)
errs = []
epochs = 100
for e in range(epochs):
    running_error=0.
    mean_running_error = 0.
    for input_batch, target_batch in train_data:
    #     print(input_batch.shape, target_batch.shape)
        feed_dict = {}
        for i in range(n_cond):
            feed_dict[encoder_inputs[i].name] = input_batch[:,i].reshape(-1,1)
        for i in range(n_pred):
            feed_dict[decoder_inputs[i].name] = input_batch[:,n_cond+i].reshape(-1,1)
        for i in range(n_pred):
            feed_dict[targets[i].name] = target_batch[:,i]
        _, err, predictions = sess.run([train_step, mae, preds], feed_dict=feed_dict)
        errs.append(err)
        mean_preds = np.mean(input_batch[:,:n_cond], axis=1).reshape(-1,1)
#         print(mean_preds.shape, target_batch.shape)
        mean_errs = np.abs(mean_preds - target_batch)
#         print(mean_errs.shape)
        batch_mean_err = np.mean(mean_errs)
        running_error += err
        mean_running_error += batch_mean_err
    running_error /= train_data.num_batches
    mean_running_error /= train_data.num_batches
    print('End of epoch {0}: running error average = {1:.3f}\n mean error average = {2:.3f}'.format(e + 1, running_error, mean_running_error))
# print(len(preds), preds[0])

31.5777953919
(1000, 490)
(1000, 38, 120)
(38000, 120, 1)
38 overlapping windows of length 120 in training date range
(38000, 120) (38000, 60)
End of epoch 1: running error average = 0.064
 mean error average = 0.055
End of epoch 2: running error average = 0.025
 mean error average = 0.055
End of epoch 3: running error average = 0.016
 mean error average = 0.055
End of epoch 4: running error average = 0.013
 mean error average = 0.055
End of epoch 5: running error average = 0.011
 mean error average = 0.055
End of epoch 6: running error average = 0.010
 mean error average = 0.055
End of epoch 7: running error average = 0.009
 mean error average = 0.055
End of epoch 8: running error average = 0.008
 mean error average = 0.055


KeyboardInterrupt: 

In [ ]:
0.043 / 0.055